In [2]:
import pandas as pd
import os
from nltk.corpus import stopwords
import nltk.data
import logging
import numpy as np  # Make sure that numpy is imported
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
import pickle

In [3]:
import sys
sys.path.append('/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/')
from KaggleWord2VecUtility import KaggleWord2VecUtility
DATADIR='/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/'

In [16]:
# Read data from files
train = pd.read_csv( os.path.join(DATADIR, 'labeledTrainData.tsv'), header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( os.path.join(DATADIR, "unlabeledTrainData.tsv"), header=0,  delimiter="\t", quoting=3 )

In [17]:
# Verify the number of reviews that were read (100,000 in total)
print "Read %d labeled train reviews " \
     "and %d unlabeled reviews\n" % (train["review"].size, unlabeled_train["review"].size )

Read 25000 labeled train reviews and 50000 unlabeled reviews



In [6]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [21]:
print train.ix[1,][2]
train.loc[0:2]

<type 'str'>


,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


### Get an even split train/test set from our labelled training set

because Kaggle's test set has no labels so we need our own labelled test set.

In [7]:
fracTrain = 0.5
nSamples = train.shape[0]
order = np.random.permutation(nSamples) # come up with a random ordering
splitIndex = int(np.round(nSamples*fracTrain))
train1 = train.ix[order[:splitIndex],:]
test1 = train.ix[order[splitIndex:],:]

In [8]:
print train1.shape
print test1.shape

(12500, 3)
(12500, 3)


In [16]:
print test1.iloc[[0]]

             id  sentiment                                             review
21130  "5706_2"          0  "This movie is like the thousand \"cat and mou...


In [15]:
pickle.dump(train1, open('train1.p', 'wb'))
pickle.dump(test1, open('test1.p', 'wb'))
pickle.dump(unlabeled_train, open('unlabeled_train.p', 'wb'))

# Word2Vec inversion

Here is some stuff to split the labelled training set into positive and negative reviews. NOTE: takes a long time to run.

Gets a list of positive review indexes and negative review indexes. Takes those respective reviews and parses them into sentences and appends to a list of positive and negative sentences, respectively. Also does the same for the unlabelled training sentences.

In [10]:
# ****** Split the labeled and unlabeled training sets into clean sentences
#
sentences_pos = []  # Initialize an empty list of sentences
sentences_neg = []  # Initialize an empty list of sentences
sentences_unlabelled = []  # Initialize an empty list of sentences

inxs_pos = np.where(train1["sentiment"] == 1)[0].tolist()
inxs_neg = np.where(train1["sentiment"] == 0)[0].tolist()

print "Parsing sentences from training set"
for inx in inxs_pos:
    review = train1["review"].iloc[inx]
    sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

for inx in inxs_neg:
    review = train1["review"].iloc[inx]
    sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)    
    
print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    sentences_unlabelled += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

Parsing sentences from training set


/home/sarahwie/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/home/sarahwie/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:198: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/home/sarahwie/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful So

Parsing sentences from unlabeled set


/home/sarahwie/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/home/sarahwie/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup

Now do the same for all sentences without splitting to build a vocabulary:

In [17]:
# ****** Split the labeled and unlabeled training sets into clean sentences
#
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train1["review"]:
    sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

Parsing sentences from training set
Parsing sentences from unlabeled set


Some stuff from Taddy's code (much simpler than Kaggle method for building Word2Vec models). 

In [18]:
from gensim.models import Word2Vec
import multiprocessing

## create a w2v learner 
basemodel = Word2Vec(
    workers=multiprocessing.cpu_count(), # use your cores
    iter=3, # iter = sweeps of SGD through the data; more is better
    hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
    )
print basemodel

Word2Vec(vocab=0, size=100, alpha=0.025)


In [19]:
basemodel.build_vocab(sentences) 

Train 2 models off of the base model (one pos & one neg).

In [20]:
from copy import deepcopy
models = [deepcopy(basemodel) for i in range(2)]
models[0].train(sentences_neg, total_examples=len(sentences_neg) )
models[1].train(sentences_pos, total_examples=len(sentences_pos) )

3301132

In [21]:
pickle.dump(models, open('models.p', 'wb'))

### Inversion of the distributed representations

At this point, we have 5 different word2vec language representations. Each 'model' has been trained conditional (i.e., limited to) text from a specific star rating. We will apply Bayes rule to go from p(text|stars) to p(stars|text).
For any new sentence we can obtain its likelihood (lhd; actually, the composite likelihood approximation; see the paper) using the score function in the word2vec class. We get the likelihood for each sentence in the first test review, then convert to a probability over star ratings. Every sentence in the review is evaluated separately and the final star rating of the review is an average vote of all the sentences. This is all in the following handy wrapper.

In [42]:
"""
docprob takes two lists
* docs: a list of documents, each of which is a list of sentences
* models: the candidate word2vec models (each potential class)

it returns the array of class probabilities.  Everything is done in-memory.
"""

import pandas as pd # for quick summing within doc

def docprob(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").mean()
    return prob

In [23]:
# read in the test set as a list of a list of words
docs = []
for review in test1["review"]:
    docs.append(KaggleWord2VecUtility.review_to_sentences(review, tokenizer))

In [31]:
print docs[0]

[[u'this', u'movie', u'is', u'like', u'the', u'thousand', u'cat', u'and', u'mouse', u'movies', u'that', u'preceded', u'it'], [u'the', u'following', u'may', u'look', u'like', u'a', u'spoiler', u'but', u'it', u'really', u'just', u'describes', u'a', u'large', u'class', u'of', u'movies', u'there', u'is', u'the', u'passionate', u'wise', u'main', u'character', u'his', u'goofy', u'but', u'well', u'meaning', u'sidekick', u'with', u'his', u'ill', u'placed', u'attempts', u'at', u'humorous', u'comments', u'the', u'initially', u'hostile', u'but', u'soon', u'softened', u'gorgeous', u'lady', u'who', u'triggers', u'the', u'inevitable', u'unlikely', u'love', u'story', u'the', u'loved', u'ones', u'taken', u'hostage', u'and', u'of', u'course', u'the', u'careless', u'evil', u'adversary', u'with', u'his', u'brutal', u'minions'], [u'everybody', u'has', u'seen', u'tons', u'of', u'these', u'movies', u'already', u'and', u'national', u'treasure', u'is', u'like', u'any', u'one', u'of', u'them', u'with', u'only'

In [26]:
probstest = docprob(docs[0:2], models)

In [27]:
probstest

,0,1
doc,,
0,0.484133,0.515867
1,0.573073,0.426927


### Test set example
As an example, we apply the inversion on the full test set.

In [40]:
# get the probs (note we give docprob our test set plus the models)
probs = docprob(docs,models).astype(object)

In [29]:
probs

,0,1
doc,,
0,0.484133,0.515867
1,0.573073,0.426927
2,0.261172,0.738828
3,0.0482365,0.951763
4,0.536646,0.463354
5,0.335723,0.664277
6,0.527615,0.472385
7,0.0064288,0.993571
8,0.594172,0.405828


In [30]:
pickle.dump(probs, open('probs.p', 'wb'))
pickle.dump(docs, open('docs.p','wb'))

A numpy array of ones for the number of docs that we're predicting:

In [32]:
predictions = np.ones((probs.shape[0]))

In [33]:
print np.shape(predictions)

(12500,)


In [34]:
predictions[np.where(probs.iloc[:,0] > 0.5)] = 0 # The first column is the negative model

In [35]:
predictions

array([ 1.,  0.,  1., ...,  0.,  0.,  1.])

In [36]:
print predictions.shape
print test1["sentiment"].shape

(12500,)
(12500,)


In [37]:
print np.size(np.where(predictions == test1["sentiment"]))*1./np.size(predictions)

0.86696


In [38]:
pickle.dump(predictions, open('predictions.p', 'wb'))

### probsentences was created by commenting out the "prob = prob.groupby("doc").mean()" line in the function and then getting probs.

In [41]:
pickle.dump(probssentences, open('probssentences.p', 'wb'))

In [44]:
probssentences[0:20]

,0,1,doc
0,0.5986,0.4014,0
1,0.00792023,0.99208,0
2,0.369248,0.630752,0
3,0.445318,0.554682,0
4,0.999581,0.000419095,0
5,0.00449226,0.995508,1
6,0.918007,0.0819928,1
7,0.55223,0.44777,1
8,0.986808,0.0131918,1
9,0.721142,0.278858,1
